# Generating_Images

### Headers

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0'
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
#import warnings
#import csv
import gc
from time import time
import logging
#from scipy.io import loadmat
#from select_win import select_win,select_windows
import load_data as ldfile
#import create_model as cmodel
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.device('/device:GPU:0')

#from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample
#from utils import hamilton_detector, christov_detector, findpeaks, engzee_detector
from ecg_detectors.ecgdetectors import Detectors, MWA, panPeakDetect, searchBack

np.random.seed(354)
#sns.set()
#warnings.filterwarnings('ignore')
#logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.random.set_seed(1342) # Set seed for reproducibility

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")


Default GPU Device:/device:GPU:0


### Loading Up Images from Dataset

In [2]:
t1 = time()
input_directory = '/home/koolaid/Documents/PN_data/WFDB_PTB' # Dataset on beastie 
train_data,train_labels_bin,val_data,val_labels_bin = ldfile.load_data2D_bin(input_directory) 
#train_dataset,val_dataset = ldfile.load_tf(train_data,train_labels_bin,val_data,val_labels_bin)
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))


total number of files:  516
Number of files for trainig:  412
Number of files for validation:  51
Number of files for testing:  51
Labels:
0. 164865005
1. 164889003
2. 164890007
3. 164895002
4. 164896001
5. 204384007
6. 266249003
7. 282825002
8. 314208002
9. 368009
10. 426783006
11. 53741008
12. 6374002
13. 698247007
14. 74390002
15. 84114007

number of test signals =  53

train_data type: <class 'list'>
train_labels  type: <class 'list'>
val_data  type: <class 'list'>
val_labels  type: <class 'list'>

train_data length 16473
train_labels length 16473
val_data length: 2016
val_labels length: 2016

train_data type: <class 'numpy.ndarray'>
train_labels_bin  type: <class 'numpy.ndarray'>
val_data  type: <class 'numpy.ndarray'>
val_labels_bin  type: <class 'numpy.ndarray'>

train_data.shape: (16473, 12, 2500)
train_labels.shape: (16473, 16)
val_data.shape: (2016, 12, 2500)
val_labels.shape: (2016, 16)

train_data.shape: (16473, 12, 2500, 1)
val_data.shape: (2016, 12, 2500, 1)
Loading data 

In [3]:
train_data.shape

(16473, 12, 2500, 1)

# Black and white Plotting original signal

### Plotting the Data from the dataset as an image

In [103]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure


def create_plot(data1, i, dir_path):
    
    #plt.figure()
    #figure(num=None, figsize=(100,10), dpi = 100)
    
    #fig,lead=plt.subplots(12,1, figsize=(20,60)) #### ORIGINAL - FOR ALL 12 LEADS
    #fig,lead=plt.subplots(4,1, figsize=(10,20), dpi = 100) ### Version 1
    fig,lead=plt.subplots(4,1, figsize=(20,40), dpi = 50) ### Version 2
    #plt.gca().set_axis_off()
    #fig.cmap=plt.cm.binary
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0) #deals with all the whitespace around the plots
    #plt.margins(0,0)

    lead[0].plot(data1[0],'white')
    lead[0].set_axis_off()
    #lead[0].cmap=plt.cm.binary
    lead[1].plot(data1[1],'white')
    lead[1].set_axis_off()
    
    #lead[2].plot(data1[2],'black')
    #lead[2].set_axis_off()
    #lead[3].plot(data1[3],'black')
    #lead[3].set_axis_off()
    #lead[1].plot(data1[4],'black')
    #lead[1].set_axis_off()
    #lead[5].plot(data1[5],'black')
    #lead[5].set_axis_off()
    #lead[2].plot(data1[6],'black')
    #lead[2].set_axis_off()
    lead[2].plot(data1[7],'white')
    lead[2].set_axis_off()
    
    #lead[8].plot(data1[8],'black')
    #lead[8].set_axis_off()
    #lead[9].plot(data1[9],'black')
    #lead[9].set_axis_off()
    lead[3].plot(data1[10],'white')
    lead[3].set_axis_off()
    
    #lead[3].plot(data1[11],'black')
    #lead[3].set_axis_off()
    
  
    
    file_name = 'img%d.png' %(i) 
    #fig.tight_layout()
    #plt.show()
    fig.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

In [104]:
# test block to test the function in step 1:
dir_path = "temp_delete"
os.makedirs(dir_path, exist_ok=True)

i = 0
j = 1



test_1 = create_plot(train_data[i], j, dir_path)

In [6]:
with tf.device("/GPU:0"):
    t1 = time()
    X_data = []
    files = sorted(glob.glob('temp_delete/*.png')) #change temp_outputs to your temporary storage area for images created
    for myFile in files:
        print(myFile)
        image = cv2.imread (myFile)
        X_data.append (image)
    train_im = np.asarray(X_data)/255


temp_delete/img1.png


In [7]:
un= np.unique(train_im)
print(un)
train_im

[0.         0.00392157 0.00784314 0.01176471 0.01568627 0.01960784
 0.02352941 0.02745098 0.03137255 0.03529412 0.03921569 0.04313725
 0.04705882 0.05098039 0.05490196 0.05882353 0.0627451  0.06666667
 0.07058824 0.0745098  0.07843137 0.08235294 0.08627451 0.09019608
 0.09411765 0.09803922 0.10196078 0.10588235 0.10980392 0.11372549
 0.11764706 0.12156863 0.1254902  0.12941176 0.13333333 0.1372549
 0.14117647 0.14509804 0.14901961 0.15294118 0.15686275 0.16078431
 0.16470588 0.16862745 0.17254902 0.17647059 0.18039216 0.18431373
 0.18823529 0.19215686 0.19607843 0.2        0.20392157 0.20784314
 0.21176471 0.21568627 0.21960784 0.22352941 0.22745098 0.23137255
 0.23529412 0.23921569 0.24313725 0.24705882 0.25098039 0.25490196
 0.25882353 0.2627451  0.26666667 0.27058824 0.2745098  0.27843137
 0.28235294 0.28627451 0.29019608 0.29411765 0.29803922 0.30196078
 0.30588235 0.30980392 0.31372549 0.31764706 0.32156863 0.3254902
 0.32941176 0.33333333 0.3372549  0.34117647 0.34509804 0.349019

array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        ...,

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]])

# butterworth lowpass filtering

In [4]:
from scipy import signal
import matplotlib.pyplot as plt

In [5]:
sig = train_data[0]
sig = sig.reshape(12,2500)
sig.shape

(12, 2500)

In [10]:
sos = signal.butter(10, 50, 'low', fs=500, output='sos')
buttered = signal.sosfilt(sos, sig)
plt.plot(buttered[1])
plt.title('After 50 Hz low-pass filter')
#plt.axis([0, 1, -2, 2])
plt.xlabel('Time [seconds]')
plt.tight_layout()
plt.show()


In [8]:
plt.plot(train_data[0][1])
plt.title('Original Signal')
#plt.axis([0, 1, -2, 2])
plt.xlabel('Time [seconds]')
plt.tight_layout()
plt.show()

# Butterworth high pass filtering

In [45]:
sos = signal.butter(10, 0.05, 'hp', fs=500, output='sos')
rebuttered = signal.sosfilt(sos, buttered)

plt.plot(rebuttered[1])
plt.title('After 0.5 Hz high-pass filter')
#plt.axis([0, 1, -2, 2])
plt.xlabel('Time [seconds]')
plt.tight_layout()
plt.show()


In [47]:
rebuttered.shape

(12, 2500)

# Remove Baseline Wandering

In [17]:
import heartpy as hp

In [18]:
def filter_and_visualise(data, sample_rate):
    '''
    function that filters using remove_baseline_wander 
    and visualises result
    '''
    
    filtered = hp.remove_baseline_wander(data, sample_rate)

    plt.figure(figsize=(12,3))
    plt.title('signal with baseline wander removed')
    plt.plot(filtered)
    plt.show()

    #And let's plot both original and filtered signal, and zoom in to show peaks are not moved
    #We'll also scale both signals with hp.scale_data
    #This is so that they have the same amplitude so that the overlap is better visible
    plt.figure(figsize=(12,3))
    plt.title('zoomed in signal with baseline wander removed, original signal overlaid')
    plt.plot(hp.scale_data(data[200:1200]))
    plt.plot(hp.scale_data(filtered[200:1200]))
    plt.show()
    
    return filtered


In [22]:
train_data[0][0].shape

(2500, 1)

In [46]:
data = train_data[0][0].reshape(2500)
sample_rate = 500

baselinewander = hp.remove_baseline_wander(buttered, sample_rate)

filtered = filter_and_visualise(rebuttered[1], sample_rate)


# import NLMDenoising as nlm

In [51]:
from scipy import signal
import matplotlib.pyplot as plt

import numpy as np
import math
import sys


def NLM_1dDarbon(signal,Nvar,P,PatchHW):
    if isinstance(P,int): # scalar has been entered; expand into patch sample index vector
        P = P-1 #Python start index from 0
        Pvec = np.array(range(-P,P+1)) #0 -> -P
    else:
        Pvec = P # use the vector that has been input
    signal = np.array(signal)
    #debug = [];
    N = len(signal)
    print(N)

    denoisedSig = np.empty(len(signal)) #NaN * ones(size(signal));
    denoisedSig[:] = np.nan
    # to simpify, don't bother denoising edges
    iStart = PatchHW+1
    iEnd = N - PatchHW
    denoisedSig[iStart: iEnd] = 0

    #debug.iStart = iStart;
    #debug.iEnd = iEnd;

    # initialize weight normalization
    Z = np.zeros(len(signal))
    cnt = np.zeros(len(signal))

    # convert lambda value to  'h', denominator, as in original Buades papers
    Npatch = 2 * PatchHW + 1
    h = 2 * Npatch * Nvar**2

    for idx in Pvec: # loop over all possible differences: s - t
        # do summation over p - Eq.3 in Darbon
        k = np.array(range(N))
        print('k:',k)
        print('idx:', idx)
        
        kplus = k + idx
        print('kplus:', kplus)
        print('kplus shape:', kplus.shape)
        
        igood = np.where((kplus >=0) & (kplus < N)) # ignore OOB data; we could also handle it
        print('igood:', igood)
        print('igood shape:', len(igood[0]))
        
        SSD = np.zeros(len(k)).reshape(2500,1)
        print('SSD:', SSD)
        print('SSD shape:', SSD.shape)
        print('k[igood]:', k[igood])
        print('kigood len:', k[igood].shape)
        print('kplus[igood]:', kplus[igood])
        print('kplusigood len:', kplus[igood].shape)
       
        print('signal k:', signal[k[igood]])#.reshape(2401,))
        print('signal k len:', signal[k[igood]].shape)
        print('signal kplus:', signal[kplus[igood]])#.reshape(2401,))
        print('signal kplus len:', signal[kplus[igood]].shape)
        
        SSD[igood] = (signal[k[igood]] - signal[kplus[igood]])**2
        print('SSD[igood]:',SSD[igood])
        
        Sdx = np.cumsum(SSD)
        print('Sdx:', Sdx)
        print('Sdx shape:', Sdx.shape)
        

        for ii in range(iStart,iEnd): # loop over all points 's'
            distance = Sdx[ii + PatchHW] - Sdx[ii - PatchHW-1] #Eq 4;this is in place of point - by - point MSE
            # but note the - 1; we want to icnlude the point ii - iPatchHW

            w = math.exp(-distance/h) # Eq 2 in Darbon
            t = ii + idx # in the papers, this is not made explicit

            if t>0 and t<N:
                denoisedSig[ii] = denoisedSig[ii] + w * signal[t]
                
                
                Z[ii] = Z[ii] + w
                
                
                #cnt[ii] = cnt[ii] + 1
                #print('ii',ii)
                #print('t',t)
                #print('w',w)
                #print('denoisedSig[ii]', denoisedSig[ii])
                #print('Z[ii]',Z[ii])
     # loop over shifts
    print('denoisedSig[ii]:', denoisedSig[ii])
    print('denoisedSig[ii] shape:', denoisedSig[ii].shape)
    
    print('Z[ii]:', Z[ii])
    print('Z[ii] shape:', Z[ii].shape)
    
    # now apply normalization
    denoisedSig = (denoisedSig/(Z + sys.float_info.epsilon)).reshape(2500,1)
    print('denoisedSig:', denoisedSig)
    print('denoisedSig shape:', denoisedSig.shape)
    
    
    print('denoisedSig 0-patchHW+1:', denoisedSig[0: PatchHW+1])
    print('denoisedSig 0-patchHW+1 shape:', denoisedSig[0: PatchHW+1].shape)
    print('signal 0-patchHW+1:', signal[0: PatchHW+1])
    print('signal 0-patchHW+1 shape:', signal[0: PatchHW+1].shape)
    
    print('denoisedSig -patchHW:', denoisedSig[- PatchHW:])
    print('denoisedSig -patchHW shape:', denoisedSig[- PatchHW:].shape)
    print('signal -patchHW:', signal[-PatchHW: ])
    print('signal -patchHW shape:', signal[-PatchHW: ].shape)
    
    denoisedSig[0: PatchHW+1] =signal[0: PatchHW+1]
    denoisedSig[ - PatchHW: ] =signal[- PatchHW: ]
    #debug.Z = Z;

    return denoisedSig#,debug

In [48]:
train_data[1][0].reshape(2500,)

array([536., 531., 515., ..., 262., 267., 271.])

In [88]:
train_data[1][0].shape
# train_data[1][0] to be fed into NLM_1dDarbon as signal

(2500, 1)

In [92]:
rebuttered = rebuttered.reshape(2500,1)
rebuttered.shape
rebuttered

array([[7.22199690e-02],
       [5.31935930e-01],
       [1.86673409e+00],
       ...,
       [7.01423430e+01],
       [7.22998647e+01],
       [7.44349151e+01]])

In [93]:
Nvar = 0.6*.02
p = 100
patchHW = 1
denoised = NLM_1dDarbon(rebuttered,Nvar, p, patchHW)

2500
k: [   0    1    2 ... 2497 2498 2499]
idx: -99
kplus: [ -99  -98  -97 ... 2398 2399 2400]
kplus shape: (2500,)
igood: (array([  99,  100,  101, ..., 2497, 2498, 2499]),)
igood shape: 2401
SSD: [[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
SSD shape: (2500, 1)
k[igood]: [  99  100  101 ... 2497 2498 2499]
kigood len: (2401,)
kplus[igood]: [   0    1    2 ... 2398 2399 2400]
kplusigood len: (2401,)
signal k: [[-170.4092809 ]
 [-170.53678167]
 [-171.0998385 ]
 ...
 [  70.14234296]
 [  72.29986471]
 [  74.43491506]]
signal k len: (2401, 1)
signal kplus: [[  0.07221997]
 [  0.53193593]
 [  1.86673409]
 ...
 [-71.84258725]
 [-70.81229843]
 [-68.77641569]]
signal kplus len: (2401, 1)
SSD[igood]: [[29063.9421389 ]
 [29264.50614262]
 [29917.43523399]
 ...
 [20159.72040667]
 [20481.09123934]
 [20509.48525723]]
Sdx: [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.07950507e+08
 2.07970988e+08 2.07991498e+08]
Sdx shape: (2500,)
k: [   0    1    2 ... 2497 2498 2499]
idx: -98
kplus: [ -98  -97

In [94]:
denoised

array([[7.22199690e-02],
       [5.31935930e-01],
       [1.86673409e+00],
       ...,
       [7.01423430e+01],
       [7.22998647e+01],
       [7.44349151e+01]])

In [7]:
import matplotlib
matplotlib.matplotlib_fname()

import matplotlib
matplotlib.use("GTK3Agg")
import matplotlib.pyplot as plt


In [97]:
denoised = denoised.reshape(2500,1)
file_name = denoised


plt.plot(denoised)
plt.title('signal with NLM Denoising')
plt.ylabel('some numbers')
plt.show()
#fig.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
plt.close()

# Black and white image of the denoised signal

In [138]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure


def create_plot(data1, i, dir_path):
    
    #plt.figure()
    #figure(num=None, figsize=(100,10), dpi = 100)
    
    #fig,lead=plt.subplots(12,1, figsize=(20,60)) #### ORIGINAL - FOR ALL 12 LEADS
    #fig,lead=plt.subplots(4,1, figsize=(10,20), dpi = 100) ### Version 1
    fig,lead=plt.subplots(1,1, figsize=(20,40), dpi = 50) ### Version 2
    #plt.gca().set_axis_off()
    #fig.cmap=plt.cm.binary
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0) #deals with all the whitespace around the plots
    #plt.margins(0,0)

    lead[0].plot(data1[0],'white')
    lead[0].set_axis_off()
    #lead[0].cmap=plt.cm.binary
    #lead[1].plot(data1[1],'white')
   #lead[1].set_axis_off()
    
    #lead[2].plot(data1[2],'black')
    #lead[2].set_axis_off()
    #lead[3].plot(data1[3],'black')
    #lead[3].set_axis_off()
    #lead[1].plot(data1[4],'black')
    #lead[1].set_axis_off()
    #lead[5].plot(data1[5],'black')
    #lead[5].set_axis_off()
    #lead[2].plot(data1[6],'black')
    #lead[2].set_axis_off()
    #lead[2].plot(data1[7],'white')
    #lead[2].set_axis_off()
    
    #lead[8].plot(data1[8],'black')
    #lead[8].set_axis_off()
    #lead[9].plot(data1[9],'black')
    #lead[9].set_axis_off()
    #lead[3].plot(data1[10],'white')
    #lead[3].set_axis_off()
    
    #lead[3].plot(data1[11],'black')
    #lead[3].set_axis_off()
    
  
    
    file_name = 'img%d.png' %(i) 
    #fig.tight_layout()
    #plt.show()
    fig.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

In [160]:
def create_plot(data1, i, dir_path):
    
    img = plt.imshow(data1, interpolation='nearest')
    img.set_cmap('binary')
    plt.axis('off')
    file_name = 'img%d.png' %(i) 
    plt.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
  
    
    #plt.figure()
    #figure(num=None, figsize=(100,10), dpi = 100)
    
    #fig,lead=plt.subplots(12,1, figsize=(20,60)) #### ORIGINAL - FOR ALL 12 LEADS
    #fig,lead=plt.subplots(4,1, figsize=(10,20), dpi = 100) ### Version 1
    fig,lead=plt.subplots(1,1, figsize=(20,40), dpi = 50) ### Version 2
    #plt.gca().set_axis_off()
    #fig.cmap=plt.cm.binary
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0) #deals with all the whitespace around the plots
    #plt.margins(0,0)

    lead[0].plot(data1[0],'white')
    lead[0].set_axis_off()
    #lead[0].cmap=plt.cm.binary
    #lead[1].plot(data1[1],'white')
   #lead[1].set_axis_off()
    
    #lead[2].plot(data1[2],'black')
    #lead[2].set_axis_off()
    #lead[3].plot(data1[3],'black')
    #lead[3].set_axis_off()
    #lead[1].plot(data1[4],'black')
    #lead[1].set_axis_off()
    #lead[5].plot(data1[5],'black')
    #lead[5].set_axis_off()
    #lead[2].plot(data1[6],'black')
    #lead[2].set_axis_off()
    #lead[2].plot(data1[7],'white')
    #lead[2].set_axis_off()
    
    #lead[8].plot(data1[8],'black')
    #lead[8].set_axis_off()
    #lead[9].plot(data1[9],'black')
    #lead[9].set_axis_off()
    #lead[3].plot(data1[10],'white')
    #lead[3].set_axis_off()
    
    #lead[3].plot(data1[11],'black')
    #lead[3].set_axis_off()
    
  
    
    file_name = 'img%d.png' %(i) 
    #fig.tight_layout()
    #plt.show()
    fig.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
    plt.close()

In [161]:
blackdenoised = denoised.reshape(1,2500)

In [162]:
# test block to test the function in step 1:
dir_path = "temp_delete"
os.makedirs(dir_path, exist_ok=True)

i = 0
j = 3

test_1 = create_plot(blackdenoised, j, dir_path)

RuntimeError: libpng signaled error

# Identifying the values in a b/w image


In [117]:

X_data = []
files = sorted(glob.glob('temp_delete/img2.png')) #change temp_outputs to your temporary storage area for images created
for myFile in files:
    print(myFile)
    image = cv2.imread (myFile)
    X_data.append (image)
train_im = np.asarray(X_data)/255


temp_delete/img2.png


In [132]:
un= np.unique(train_im)
#print(un)
train_im.shape

(1, 2010, 1021, 3)

# Thresholding an image

In [164]:
img = cv2.imread('temp_delete/img2.png',0)
un2 = np.unique(img)
print (un2)

[0]


In [165]:
img = cv2.imread('temp_delete/img2.png',0)
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

titles = ['Original Image','BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [img, thresh1, thresh2, thresh3, thresh4, thresh5]
plt.imshow(images[1],'gray')

#for i in range(6):
#    plt.subplot(2,3,i+1),plt.imshow(images[i],'gray')
#    plt.title(titles[i])
#    plt.xticks([]),plt.yticks([])

plt.show()

In [137]:
import cv2

img = cv2.imread('temp_delete/img2.png',0)
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
#ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
#ret,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
#ret,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
#ret,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

plt.imshow(thresh1,'gray')
plt.title('BW/binary image')
plt.show()
#fig.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
plt.close()


# Other wasted attempt

In [77]:
#denoised = denoised.reshape(2500,1)
#file_name = denoised

import matplotlib
matplotlib.use("GTK3Agg")
import matplotlib.pyplot as plt

plt.plot(train_data[1][0])
plt.ylabel('some numbers')
plt.show()
#fig.savefig(os.path.join(dir_path,file_name), facecolor ='black', transparent = None, bbox_inches = 'tight', pad_inches = 0)
plt.close()

STEPS:

1. Create an output folder (test to see if one exists)
2. Loop through 'train_data' -> plot images (do only 5000 at a time)

3. Save image into output folder
4. Take plotted image, open it and read it with imread (as an array item) 
5. append it to an array 'train_im'

6. Read through 'train_labels_bin' and copy section (5000 at a time) into 'train_lab'

7. Repeat process for 'val_data' and 'val_labels_bin'

8. Make train_dataset and val_dataset from data and labels combined.

### TRAINING DATA

#### **ERROR** - Separate 'train_batch' into 2 functions and break the loop for automating into two parts

In [5]:
def train_batch (train_data, start, end):
    
    # test block to test the function in step 1:
    dir_path = "train_outputs_v1"
    os.makedirs(dir_path, exist_ok=True)

    diff = end-start
    range_end = 1000+diff
    # test block to test the loop in step 2 & 3:
    for i,j in zip (range(start, end), range(10000,range_end)):  #the second range is for the file numbers you're gonna generate in the 
        test_1 = create_plot(train_data[i], j, dir_path)
    

    # test blcok to test saving the img in step 4 & 5:
    X_data = []
    files = sorted(glob.glob('train_outputs_v1/*.png')) #change temp_outputs to your temporary storage area for images created
    for myFile in files:
        image = cv2.imread (myFile)
        X_data.append (image)
    #train_im = np.asarray(X_data)
    
    #return train_im
    return X_data

In [ ]:
# Tracking for Training Data:

#Train_data size: 26828
#single 'train_batch': 500
#number of batches to load: 53.656

t1 = time()

#train_d1 = [[]]
for i in range(0,50):
    length = 1000 # pick how big do you want your length to be
    start = i*length
    end = start+length
    print('round:', i, 'start:', start, 'end:', end)
    t_d = train_batch(train_data, start, end)
    print('converting into array for round:', i)
    if i==0:
        train_d = np.asarray(t_d)
        #print('starting to work')
    else:
        train_d1 = np.asarray(t_d)
        #print('is it working?')
        train_d = np.append(train_d, train_d1 , axis=0) 
    
    del t_d
    gc.collect()

print(train_d.shape)
    
np.save('train_v1.npy', train_d)
del train_d, train_d1
gc.collect()
    
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))   

round: 0 start: 0 end: 1000
converting into array for round: 0
round: 1 start: 1000 end: 2000
converting into array for round: 1
round: 2 start: 2000 end: 3000
converting into array for round: 2


In [ ]:
## DO NOT DO THIS UNTIL ALL 26828  are loaded in train_set_#.npy files
#del train_data
#gc.collect()

### VALIDATION DATA

In [8]:
def val_batch (val_data, start, end):
    

    # test block to test the function in step 1:
    dir_path = "val_outputs_v1"
    os.makedirs(dir_path, exist_ok=True)

    # test block to test the loop in step 2 & 3:
    diff = end-start
    range_end = 1000+diff
    for i,j in zip (range(0,1000), range(10000,range_end)): #the second range is for the file numbers you're gonna generate in the 
        test_1 = create_plot(val_data[i], j, dir_path)

    # test blcok to test saving the img in step 4 & 5:
    X_data = []
    files = sorted(glob.glob('val_outputs_v1/*.png')) #change temp_outputs to your temporary storage area for images created
    for myFile in files:
        
        image = cv2.imread (myFile)
        X_data.append (image)
    #val_im = np.asarray(X_data)

    
    #return val_im
    return X_data

In [9]:
# Tracking for Validation Data:

#Train_data size: 3532
#single 'train_batch': 500
#number of batches to load: ~6

t1 = time()

#train_d1 = [[]]
for i in range(0,3):
    length = 1000 # pick how big do you want your length to be
    start = i*length
    end = start+length
    print('round:', i, 'start:', start, 'end:', end)
    v_d = val_batch(val_data, start, end)
    print('converting into array for round:', i)
    if i==0:
        val_d = np.asarray(v_d)
    else:
        val_d1 = np.asarray(v_d)
        val_d = np.append(val_d, val_d1 , axis=0) 
    
    del v_d
    gc.collect()
    
print(val_d.shape)
    
np.save('val_v1.npy', val_d)
del val_d, val_d1
gc.collect()
    
t2 = time()
print('Loading data in {} seconds.'.format(round(t2-t1)))   

round: 0 start: 0 end: 100
converting into array for round: 0
round: 1 start: 100 end: 200
converting into array for round: 1
Loading data in 150 seconds.


In [2]:
## DO NOT DO THIS UNTIL ALL 26828  are loaded in train_set_#.npy files
#del val_data
#gc.collect()